In [44]:
import pandas as pd
import numpy as np

RANDOM_STATE = 404

In [45]:
df = pd.read_csv('../data/cardio_train.csv', delimiter=';')
df.drop(columns=['id'], inplace=True)
df.head()

,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,18857,1,165,64.0,130,70,3,1,0,0,0,1
3,17623,2,169,82.0,150,100,1,1,0,0,1,1
4,17474,1,156,56.0,100,60,1,1,0,0,0,0


In [46]:
X = df.drop(columns=['cardio'])
y = df['cardio']

In [47]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y) 
y

array([0, 1, 1, ..., 1, 1, 0], dtype=int64)

In [48]:
from sklearn.preprocessing import StandardScaler, LabelEncoder

# Select columns to be scaled
numeric_columns = ['age', 'height', 'weight', 'ap_hi', 'ap_lo', 'gender', 'cholesterol']
categorical_columns = ['gluc', 'smoke', 'alco', 'active']

# Fit and transform your data (only for numeric columns)
scaler = StandardScaler()
X[numeric_columns] = scaler.fit_transform(X[numeric_columns])

# Apply one-hot encoding to categorical columns
label_encoder = LabelEncoder()
for col in categorical_columns:
    X[col] = label_encoder.fit_transform(X[col])

X.head(10)

,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active
0,-0.436062,1.364055,0.443452,-0.847873,-0.122182,-0.088238,-0.539322,0,0,0,1
1,0.307686,-0.733108,-1.018168,0.749831,0.072610,-0.035180,2.400793,0,0,0,1
2,-0.247997,-0.733108,0.078047,-0.708942,0.007679,-0.141297,2.400793,0,0,0,0
3,-0.748152,1.364055,0.565254,0.541435,0.137541,0.017879,-0.539322,0,0,0,1
4,-0.808543,-0.733108,-1.018168,-1.264666,-0.187113,-0.194356,-0.539322,0,0,0,0
5,0.991043,-0.733108,-1.627176,-0.500546,-0.057251,-0.088238,0.930735,1,0,0,0
6,1.071700,-0.733108,-0.896366,1.305554,0.007679,-0.088238,2.400793,0,0,0,1
7,1.262602,1.364055,1.661469,1.444485,0.007679,-0.035180,2.400793,2,0,0,1
8,-0.729913,-0.733108,-0.774565,-0.222685,-0.122182,-0.141297,-0.539322,0,0,0,1
9,0.147993,-0.733108,-0.043755,-0.431081,-0.122182,-0.194356,-0.539322,0,0,0,0


In [49]:
# Dictionary to store models and their results
models_dict = {}

In [50]:
from sklearn.model_selection import train_test_split

# Split into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE)

### Random Forest classifier

In [51]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

rfModel = RandomForestClassifier(random_state=RANDOM_STATE)

rfModel.fit(X_train, y_train)

rf_pred = rfModel.predict(X_test)

rf_accuracy = metrics.accuracy_score(y_test, rf_pred)*100

models_dict['Random Forest'] = {'model': rfModel, 'accuracy': rf_accuracy}

print(f"Accuracy without CV: {rf_accuracy:.2f}")

Accuracy without CV: 71.80


In [52]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

param_grid = {
    'n_estimators': [100, 200, 300, 500],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10, 20],
    'min_samples_leaf': [1, 2, 4, 8],
    'max_features': ['sqrt', 'log2', None],
}

rf_best_params = {
    'n_estimators': [100],
    'max_depth': [10],
    'min_samples_split': [10],
    'min_samples_leaf': [1],
    'max_features': [None],
}

rf_gridsearch = GridSearchCV(estimator=rfModel,param_grid=rf_best_params, cv=5, scoring='accuracy',n_jobs=-1)
rf_gridsearch.fit(X_train, y_train)

rf_best_params = rf_gridsearch.best_params_
rf_best_estimator = rf_gridsearch.best_estimator_
rf_pred_CV = rf_best_estimator.predict(X_test)
rf_accuracy_cv = metrics.accuracy_score(y_test, rf_pred_CV)*100
classification_report_str = classification_report(y_test, rf_pred_CV, digits=4)

models_dict['Random Forest CV'] = {'model': rf_best_estimator, 'accuracy': rf_accuracy_cv}

print(f"Best Accuracy: {rf_accuracy_cv:.2f}")
print("Classification Report for RF with CV:\n", classification_report_str)

Best Accuracy: 73.47
Classification Report for RF with CV:
               precision    recall  f1-score   support

           0     0.7236    0.7647    0.7435      7041
           1     0.7474    0.7044    0.7253      6959

    accuracy                         0.7347     14000
   macro avg     0.7355    0.7345    0.7344     14000
weighted avg     0.7354    0.7347    0.7345     14000



### MLP

In [53]:
from sklearn.neural_network import MLPClassifier

mlpModel = MLPClassifier(random_state=1)
mlpModel.fit(X_train, y_train)
mlp_pred = mlpModel.predict(X_test)

mlp_accuracy = metrics.accuracy_score(y_test, mlp_pred)*100

models_dict['MLP'] = {'model': mlpModel, 'accuracy': mlp_accuracy}

print(f"Accuracy without CV: {mlp_accuracy:.2f}")

Accuracy without CV: 73.58


In [54]:
mlp_best_params = {
    'activation': ['tanh'],
    'alpha': [0.01],
    'hidden_layer_sizes': [(50, 50)],
    'max_iter': [300],
    'solver': ['adam'],
}

mlp_gridsearch = GridSearchCV(estimator=mlpModel, param_grid=mlp_best_params, cv=5, scoring='accuracy', n_jobs=-1)
mlp_gridsearch.fit(X_train, y_train)

mlp_best_estimator = mlp_gridsearch.best_estimator_
mlp_pred_CV = mlp_best_estimator.predict(X_test)
mlp_accuracy_cv = metrics.accuracy_score(y_test, mlp_pred_CV)*100
classification_report_str = classification_report(y_test, mlp_pred_CV, digits=4)

models_dict['MLP CV'] = {'model': mlp_best_estimator, 'accuracy': mlp_accuracy_cv}

print(f"Best Accuracy: {mlp_accuracy_cv:.2f}")  
print("Classification Report for MLP with CV:\n", classification_report_str)

Best Accuracy: 73.64
Classification Report for MLP with CV:
               precision    recall  f1-score   support

           0     0.7236    0.7701    0.7461      7041
           1     0.7512    0.7024    0.7260      6959

    accuracy                         0.7364     14000
   macro avg     0.7374    0.7362    0.7360     14000
weighted avg     0.7373    0.7364    0.7361     14000



### SVM

In [55]:
from sklearn.svm import SVC

svmModel = SVC(random_state=1)
svmModel.fit(X_train, y_train)
svm_pred = mlpModel.predict(X_test)

svm_accuracy = metrics.accuracy_score(y_test, svm_pred)*100

models_dict['SVM'] = {'model': svmModel, 'accuracy': svm_accuracy}

print(f"Accuracy without CV: {svm_accuracy:.2f}")

Accuracy without CV: 73.58


In [56]:
svm_param_grid = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf', 'poly'],
    'gamma': ['scale', 'auto'],
}

svm_best_params = {
    'C': [10],
    'kernel': ['rbf'],
    'gamma': ['auto'],
}

svm_gridsearch = GridSearchCV(estimator=svmModel, param_grid=svm_best_params, cv=5, scoring='accuracy', n_jobs=-1)

svm_gridsearch.fit(X_train, y_train)

svm_best_params = svm_gridsearch.best_params_
svm_best_estimator = svm_gridsearch.best_estimator_
svm_pred_CV = svm_best_estimator.predict(X_test)
svm_accuracy_cv = metrics.accuracy_score(y_test, svm_pred_CV)*100
classification_report_str = classification_report(y_test, svm_pred_CV, digits=4)

models_dict['SVM CV'] = {'model': svm_best_estimator, 'accuracy': svm_accuracy_cv}

print(f"Best Accuracy: {svm_accuracy_cv:.2f}")
print("Classification Report for SVM with CV:\n", classification_report_str)

Best Accuracy: 73.29
Classification Report for SVM with CV:
               precision    recall  f1-score   support

           0     0.7263    0.7526    0.7392      7041
           1     0.7402    0.7130    0.7263      6959

    accuracy                         0.7329     14000
   macro avg     0.7332    0.7328    0.7328     14000
weighted avg     0.7332    0.7329    0.7328     14000



### NN

In [57]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

# Initialize a sequential model
model = Sequential()

# Add input layer
model.add(Dense(units=64, activation='relu', input_dim=X_train.shape[1]))
model.add(Dense(units=32, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(units=1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test)

models_dict['NN'] = {'model': model, 'accuracy': test_accuracy*100}

print(f"Test Accuracy: {test_accuracy:.4f}")

Epoch 1/10
1400/1400 [==============================] - 3s 2ms/step - loss: 0.5945 - accuracy: 0.6936 - val_loss: 0.5660 - val_accuracy: 0.7307
Epoch 2/10
1400/1400 [==============================] - 3s 2ms/step - loss: 0.5608 - accuracy: 0.7297 - val_loss: 0.5478 - val_accuracy: 0.7373
Epoch 3/10
1400/1400 [==============================] - 3s 2ms/step - loss: 0.5523 - accuracy: 0.7304 - val_loss: 0.5452 - val_accuracy: 0.7398
Epoch 4/10
1400/1400 [==============================] - 2s 2ms/step - loss: 0.5497 - accuracy: 0.7316 - val_loss: 0.5411 - val_accuracy: 0.7387
Epoch 5/10
1400/1400 [==============================] - 2s 2ms/step - loss: 0.5471 - accuracy: 0.7334 - val_loss: 0.5506 - val_accuracy: 0.7354
Epoch 6/10
1400/1400 [==============================] - 2s 2ms/step - loss: 0.5475 - accuracy: 0.7338 - val_loss: 0.5413 - val_accuracy: 0.7393
Epoch 7/10
1400/1400 [==============================] - 2s 2ms/step - loss: 0.5460 - accuracy: 0.7331 - val_loss: 0.5415 - val_accuracy:

### Comparison of results

In [58]:
for key, value in models_dict.items():
    print(f"{key}: {value['accuracy']:.2f}")    

Random Forest: 71.80
Random Forest CV: 73.47
MLP: 73.58
MLP CV: 73.64
SVM: 73.58
SVM CV: 73.29
NN: 73.58


In [59]:
from joblib import dump

# Select model with best accuracy
best_result = max(models_dict, key=lambda x: models_dict[x]['accuracy'])
best_model = models_dict[best_result]['model']
print(f"Best model: {best_result} with accuracy: {models_dict[best_result]['accuracy']:.2f}")

# Save best model to a file
dump(model, 'cardio_classifier.h5')

Best model: MLP CV with accuracy: 73.64


['cardio_classifier.h5']